<a href="https://colab.research.google.com/github/ShinAsakawa/xerion/blob/master/notebooks/xerion_demo001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Xerion, data management for simulating SM89 and PMSP96

for [wbai handson](https://wba-initiative.org/3411/)

<div>
    <center><img src="https://wba-initiative.org/wp-content/uploads/2015/05/logo.png" style="width:29%"></center>
</div>
    

<p style='text-align:center;'>
    <font color='green' size='+1' style='font-weight:bold;'>Shin Asakawa &lt;asakawa@ieee.org&gt;</font>
</p>

---
# 手順1. `Git clone` する

In [0]:
# numpy の不都合によりバージョンを変更します
!pip install --user -U numpy==1.16.0

---
- 上のセルを実行後，最後に表示される  `RESTART RUNTIME` ボタンをクリックしてください
- その後下のセルの実行に進んでください

In [0]:
# GitHub から必要なパッケージを入手してください
!git clone https://github.com/ShinAsakawa/xerion.git

In [0]:
# インストール結果を確認します
import numpy as np
from xerion.xerion import Xerion

dataset = Xerion()
#dataset.descr()      # 行頭の # を外して本セルを実行すると xerion の諸元を表示します
#dataset.usage()      # 行頭の # を外して本セルを実行すると簡単なサンプルコードを表示します

---
# 手順2. データの読み込み

In [0]:
dataset.grapheme   # 入力単語を表示します
dataset.phoneme    # 出力，単語の読みの ARPABET 表現です
dataset.seq        # オリジナルのデータ系列番号です
dataset.freq       # 単語の頻度情報です
dataset.inputs     # 入力データの三連表現
dataset.outputs    # 出力データの三連表現

---

# 手順3. 読み込んだデータの確認

ここに `input`, `output`, `freq`, `grapheme`, `phoneme`, `seq`, `tag` でアクセスできるようにしてあります。

- input: np.ndarrray((2998, 105), dtype=float32) # 文字単語のトリプレット表現 105 次元ベクトル
- output: np.ndarray((2998, 61), dtype=float32)  # 対応する音韻トリプレット表現 61 次元ベクトル
- frep: np.ndarray((2998,), dtype=float32)       # 対応する頻度情報
- seq: 
- grapheme: 入力文字列リスト
- phoneme: 出力音韻列リスト ARPABET 表現
- tag: 無視してください

SM89, PMSP96 では意味層は実装されていません。<br>
以下に例を示します

In [0]:
#以下に 3 連表現を示します。PMSP96論文中では triplet と呼ばれていたオンセット onset，母音 vowel, コーダ coda です。
for item in [dataset.Orthography, dataset.Phonology]:
    for entry in ['onset', 'vowel', 'coda']:
        print(item[entry])

## 同形異音語について

In [0]:
grapheme_set = set(dataset.grapheme)
len(grapheme_set)

print('データ総数:{0} とユニークなデータ数:{1} との差が同形異音語数になります'.format(
    len(dataset.grapheme),len(grapheme_set)))

In [0]:
# 実際に同形異音語を表示してみましょう
prev_word = ""
n_homographs = 0
for word in sorted(dataset.grapheme):
    if word == prev_word:
        n_homographs += 1
        print('{0}:{1}'.format(n_homographs, word))
    prev_word = word
print('\nn_homographs={}'.format(n_homographs)) 

---
# データないに登録された各単語の発音を調べる

- pyhon で自然言語処理する際に使われる `nltk` を使って単語の発音を調べてみます。
- `nltk` については [https://www.nltk.org/](https://www.nltk.org/) をご覧ください

In [0]:
# Colaboratory では cmudict がダウンロードされいないようなのでダウンロードします
# 一度だけ実行すればよいので，その都度実行する必要はありません
import nltk
nltk.download('cmudict')

In [0]:
from nltk.corpus import cmudict
arpabet = cmudict.dict()

# 1. Xerion の単語を取り出します
words = dataset.grapheme

# 2. Xerion に存在する単語のうち cmudict に発音の登録のない単語を抜き出します。
not_in_arpabet = [word for word in words if not word in arpabet]

print('{0}/{1}={2:.2f} percent words in Xerion data were not registered in ARPABET.'
      .format(len(not_in_arpabet), len(words), len(not_in_arpabet)/len(words)*100))

# 随分ありますね。
print(not_in_arpabet)

In [0]:
# ちなみに cmu 辞書についてですが
print(len(arpabet))

arpabet_vocab = [v for v in arpabet.keys()]
print(arpabet_vocab[:5])
arpabet_sounds = [s for s in arpabet.values()]
print(arpabet_sounds[:5])
#上記の表記については ARPABET をご覧ください

---
# ARPABET に登録されていない単語の処理

In [0]:
from itertools import product as iterprod
#arpabet = nltk.corpus.cmudict.dict()

def wordbreak(s):
    """
    See https://stackoverflow.com/questions/33666557/get-phonemes-from-any-word-in-python-nltk-or-other-modules

    """
    s = s.lower()
    if s in arpabet:
        return arpabet[s]
    middle = len(s)/2
    partition = sorted(list(range(len(s))), key=lambda x: (x-middle)**2-x)
    for i in partition:
        pre, suf = (s[:i], s[i:])
        if pre in arpabet and wordbreak(suf) is not None:
            return [x+y for x,y in iterprod(arpabet[pre], wordbreak(suf))]
    return None

# nltk の cmudict に存在しない単語の読みを表示
for word in not_in_arpabet:
    print(word, wordbreak(word))

---
# 意味層の実装

from <https://fasttext.cc/docs/en/pretrained-vectors.html>

In [0]:
# fastText データの読み込み。次の行頭の # を外してください
#!time wget https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.en.vec

In [0]:
word2vec_file = 'wiki.en.vec'
with open(word2vec_file, 'r') as f:
  lines = f.readlines()

semantics_wiki = {}
word_list = list(dataset.grapheme)    

In [0]:
%%timeit
for line in lines:    
    buff = line.strip().split(' ')
    word = buff[0]
    if word in word_list:
        semantics_wiki[word] = np.asarray([float(x) for x in buff[1:]],dtype=np.float32)

In [0]:
semantics = np.zeros((len(dataset.grapheme), 300), dtype=np.float32)

In [0]:
%%timeit
for i, w in enumerate(dataset.grapheme):
    semantics[i] = semantics_wiki[w]

In [0]:
import pickle
pkl_filename = 'SM-nsyl-fastText-win-en-vec.pkl'
with open(pkl_filename, 'wb') as f:
    pickle.dump(semantics,f)

In [0]:
!wget https://github.com/ShinAsakawa/xerion/raw/master/data/SM-nsyl-fastText-win-en-vec.pkl

In [0]:
import pickle
pkl_filename = 'SM-nsyl-fastText-win-en-vec.pkl'
with open(pkl_filename, 'rb') as f:
    semantics = pickle.load(f)

In [0]:
type(semantics), semantics.shape   # 意味層データの確認

In [0]:
# データの確認のためにデータサイズを印字します
dataset.sememe = semantics
for modality in [dataset.grapheme, dataset.phoneme, dataset.sememe]:
    len(modality), type(modality), modality.shape

In [0]:
dataset.usage()      # 基本的な使い方を表示させて以下の確認に用います

In [0]:
# 上で表示されたコードに基づいて `sklearn` のニューラルネットワークで予測してみます
from sklearn.neural_network import MLPRegressor

dataset = Xerion()
X = dataset.inputs    
y = dataset.outputs
# 本例では入力データが grapheme 三連表現，出力データが phoneme 三連表現です
model = MLPRegressor()
model.fit(X, y)
model.score(X, y)

In [0]:
# 意味層から出力表現への回帰
X = semantics
y = dataset.outputs
model.fit(X,y)
model.score(X, y)

In [0]:
# 入力データは grapheme 三連表現と fastText wikipedia の連接 105 次元 +300 次元 = 405 次元
X = dataset.inputs
X = np.concatenate((X, semantics), axis=1)
print('X.shape={}'.format(X.shape))
model.fit(X, y)
model.score(X,y)

## 非単語によるモデルの検証

In [0]:
_ = dataset.make_all()   # 検証データを作成します

In [0]:
## 作成したデータ名を表示
for i, db in enumerate(dataset.dbs.keys()):
    print('{0}:{1}'.format(i,db))

In [0]:
# ここでは原著論文と同じく Glushko の非単語リストを用います
test_data = dataset.dbs['glushkoNW-nsyl']
X_test, y_test = test_data['inputs'], test_data['outputs']
X_test.shape, X_test.dtype, y_test.shape, y_test.dtype

### ちなみに Glushko の非単語リストとは以下の文献になります
- year:1979
- title: The Organization and Activation of Orthographic Knowledge in Reading Aloud
- journal: Journal of Experimental Psychology: Human Perception and Performance, Volume 5, No. 4, pages 674-691.

健常者の単語の読みの反応潜時実験を以下に紹介します

In [0]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

rt = [529, 546, 550]
rt_errors = [0.05, 2.9, 8.3]
rt_labels = ['reg/consist','reg/inconsist','except'] #['規則/一貫', '規則/非一貫', '例外語']

plt.ylim(500,570)
plt.xlabel('word type')
plt.ylabel('latency(ms)')
plt.title('from Glushko (1979) Tab. 3')
plt.bar(x=(0,1,2), height=rt, tick_label=rt_labels)
plt.show()
# 以下は単語型別の反応潜時です

In [0]:
# 続いて単語型別のエラー率です
plt.ylim(0,10)
plt.xlabel('word type')
plt.ylabel('error ratio (%)')
plt.title('from Glushko (1979) Tab. 3')
plt.bar(x=(0,1,2), height=rt_errors, tick_label=rt_labels)
plt.show()

In [0]:
# データの次元を確認しておきます
X_train, y_train = dataset.inputs, dataset.outputs
X_test, y_test = dataset.dbs['glushkoNW-nsyl']['inputs'], dataset.dbs['glushkoNW-nsyl']['outputs']
for x in [X_train, y_train, X_test, y_test]:
    print(x.shape, x.dtype)

In [0]:
model.fit(X_train, y_train)   # もう一度訓練しておきます
model.score(X_train, y_train) # 訓練データでの性能を表示
model.score(X_test, y_test)   # 非単語リストでの性能の表示

---
- Glushko(1979) の非単語リストは非単語だけに意味系の寄与がありません
- ですので以上になります